In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import requests
from datetime import datetime
from io import BytesIO
from reportlab.lib.pagesizes import A4
from reportlab.pdfgen import canvas
from reportlab.lib.units import cm
from reportlab.lib.utils import ImageReader
from reportlab.lib import colors
import textwrap

# 🔧 사용자 설정
CSV_PATH = "/content/waste_collection_realistic_final.csv"
api_key = "여기에 실제 키 붙여넣기"

# 데이터 로드
df = pd.read_csv(CSV_PATH, parse_dates=["Collected At"])

# 분석 함수
def analyze_store_range_summary(store_name, start_date, end_date, df):
    start = pd.to_datetime(start_date)
    end = pd.to_datetime(end_date)
    store_df = df[df["Store Name"] == store_name]
    filtered = store_df[(store_df["Collected At"] >= start) & (store_df["Collected At"] <= end)]
    if filtered.empty:
        return None, None

    interval = filtered["Collected At"].diff().dt.days.mean()
    summary = {
        "store_name": store_name,
        "start": start_date,
        "end": end_date,
        "total_count": len(filtered),
        "avg_volume": round(filtered["Volume (L)"].mean(), 1),
        "avg_interval": round(interval, 1) if not pd.isna(interval) else 0,
        "total_price": int(filtered["Total Price (KRW)"].sum())
    }
    return summary, filtered

# 프롬프트 함수
def generate_prompt(summary):
    return f"""
Write a business-style executive summary for waste oil collection.

Store: {summary['store_name']}
Period: {summary['start']} ~ {summary['end']}

- Total collections: {summary['total_count']}
- Avg volume: {summary['avg_volume']} L
- Avg interval: {summary['avg_interval']} days
- Total price: {summary['total_price']} KRW

Summarize this with clear insights.
"""

def generate_chart_prompt(store_name, dates, values, ylabel):
    date_str = dates.dt.strftime("%Y-%m-%d").tolist()
    value_str = values.round(2).tolist()
    return f"""
The following graph represents {ylabel} trend for {store_name}:

X-axis (date): {date_str}
Y-axis ({ylabel}): {value_str}

Write 3–5 business insights about this graph.
"""

# Gemini API 호출
def call_gemini(prompt, api_key):
    url = "https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-pro:generateContent"
    headers = {"Content-Type": "application/json"}
    body = {"contents": [{"role": "user", "parts": [{"text": prompt}]}]}
    response = requests.post(f"{url}?key={api_key}", headers=headers, json=body)
    response.raise_for_status()
    return response.json()["candidates"][0]["content"]["parts"][0]["text"]

# 그래프 생성
def create_plot(df, store_name, y_column, title, ylabel, color):
    plt.figure(figsize=(7, 4))
    plt.plot(df["Collected At"], df[y_column], marker="o", color=color)
    plt.title(f"{title} - {store_name}")
    plt.xlabel("Date")
    plt.ylabel(ylabel)
    plt.xticks(rotation=45)
    plt.tight_layout()
    buf = BytesIO()
    plt.savefig(buf, format="png")
    buf.seek(0)
    plt.close()
    return buf

# PDF 생성 함수
def create_final_pdf(report_text, chart_texts, charts, filename, summary):
    c = canvas.Canvas(filename, pagesize=A4)
    width, height = A4

    # Header
    c.setFont("Helvetica-Bold", 16)
    c.setFillColor(colors.HexColor("#2E5C87"))
    c.drawString(2 * cm, height - 2.5 * cm, "■ ECOGEM Waste Oil Collection Report")
    c.setFillColor(colors.black)
    c.setFont("Helvetica", 11)
    c.drawString(2 * cm, height - 3.2 * cm, f"Store: {summary['store_name']}")
    c.drawString(2 * cm, height - 3.8 * cm, f"Period: {summary['start']} ~ {summary['end']}")

    # Executive Summary
    y_offset = height - 5 * cm
    c.setFont("Helvetica-Bold", 12)
    c.drawString(2 * cm, y_offset, "Executive Summary")
    y_offset -= 0.6 * cm

    c.setFont("Helvetica", 10)
    for line in textwrap.wrap(report_text, 105):
        c.drawString(2 * cm, y_offset, line)
        y_offset -= 0.45 * cm

    for i, (chart_buf, chart_text, title) in enumerate(zip(charts, chart_texts, ["Volume Trend", "Revenue Trend"])):
        y_offset -= 0.5 * cm
        c.drawImage(ImageReader(chart_buf), 2 * cm, y_offset - 6 * cm, width=16 * cm, height=6 * cm)
        y_offset -= 6.7 * cm

        c.setFont("Helvetica-Bold", 11)
        c.drawString(2 * cm, y_offset, f"■ {title} Interpretation by Gemini")
        y_offset -= 0.5 * cm

        c.setFont("Helvetica", 9)
        for para in chart_text.strip().split("\n"):
            for line in textwrap.wrap(para.strip(), 105):
                c.drawString(2 * cm, y_offset, line)
                y_offset -= 0.4 * cm
        y_offset -= 0.6 * cm
        if i == 0:
            c.showPage()
            y_offset = height - 2.5 * cm

    # Footer
    c.setFont("Helvetica", 8)
    c.setFillColor(colors.HexColor("#777777"))
    c.drawString(2 * cm, 1.5 * cm, "Prepared by ECOGEM · 2025")
    c.drawRightString(width - 2 * cm, 1.5 * cm, "© 2025 ECOGEM. All rights reserved.")
    c.save()

# 실행 설정
store_name = "Bibimbap Bros"
start_date = "2023-07-01"
end_date = "2024-02-01"

summary, filtered_df = analyze_store_range_summary(store_name, start_date, end_date, df)

if summary:
    report_text = call_gemini(generate_prompt(summary), api_key)
    volume_text = call_gemini(generate_chart_prompt(store_name, filtered_df["Collected At"], filtered_df["Volume (L)"], "Volume (L)"), api_key)
    revenue_text = call_gemini(generate_chart_prompt(store_name, filtered_df["Collected At"], filtered_df["Total Price (KRW)"], "Revenue"), api_key)

    volume_chart = create_plot(filtered_df, store_name, "Volume (L)", "Volume Trend", "Volume (L)", "blue")
    revenue_chart = create_plot(filtered_df, store_name, "Total Price (KRW)", "Revenue Trend", "KRW", "green")

    filename = f"{store_name.replace(' ', '_')}_{start_date}_to_{end_date}_report.pdf"
    create_final_pdf(report_text, [volume_text, revenue_text], [volume_chart, revenue_chart], filename, summary)
    print("\u2705 Report generated:", filename)
else:
    print("\u274c No data found for the given condition.")